## Accuracy Linearized FOM

Environment

In [1]:
import sys
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")

from ronek import env
env.set(
  device="cpu",
  device_idx=0,
  nb_threads=8,
  floatx="float64"
)

Import libraries

In [2]:
import os
import numpy as np
import scipy as sp
import joblib as jl

from tqdm import tqdm
from ronek import utils
from ronek import postproc as pp
from ronek.systems import TASystem

In [3]:
import matplotlib as mpl
# Use custom style
# See: https://matplotlib.org/1.5.3/users/style_sheets.html
path_to_styles = "/home/zanardi/Workspace/Research/styles/matplotlib/"
style_name = "paper_1column"
mpl.style.use(f"{path_to_styles}/{style_name}.mplstyle")

Define inputs

In [4]:
T = 1e4
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/RVC_O3/database/"
path_to_data = "/home/zanardi/Codes/ML/RONEK/run/RVC_O3/test/data/"
path_to_saving = "./lin_fom/"
nb_samples = 2
nb_workers = 10

In [5]:
os.makedirs(path_to_saving, exist_ok=True)

Initialize isothermal master equation model

In [6]:
system = TASystem(
  T=T,
  rates=path_to_dtb + "/kinetics.hdf5",
  species={
    k: path_to_dtb + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  use_einsum=False,
  use_factorial=True
)

Solve linear FOM

In [7]:
def compute_linfom_err(path, index, eval_err=True):
  # Load data
  data = utils.load_case(path=path, index=index)
  n0, t, n = [data[k] for k in ("n0", "t", "n")]
  rho = data["mu"][-1]
  # Solve linear system
  nl = system.solve_lin_fom(t, n0)
  if eval_err:
    # Convert to mass fractions
    w = system.mix.get_w(np.vstack(n), rho)
    wl = system.mix.get_w(np.vstack(nl), rho)
    # Compute error
    err = utils.l2_relative_error(w, wl, axis=0)
    return index, t, err
  else:
    return index, t, n, nl

In [ ]:
err = jl.Parallel(nb_workers)(
  jl.delayed(compute_linfom_err)(
    path=path_to_data,
    index=i
  ) for i in tqdm(
    iterable=range(nb_samples),
    ncols=80,
    file=sys.stdout
  )
)
indices, t, err = list(zip(*err))
err = np.vstack(err)
err_mean = np.mean(err, axis=0)

Plot relative $L^2$ error

In [9]:
pp.plot_err_ci_evolution(
  x=t[0],
  mean=err_mean,
  sem=sp.stats.sem(err, axis=0),
  size=len(err),
  alpha=0.95,
  xlim=None,
  ylim=None,
  hline=None,
  labels=[r"$t$ [s]", r"Relative $L^2$ Error"],
  scales=["log", "linear"],
  legend_loc="best",
  figname=path_to_saving+"/rel_err.pdf",
  save=True,
  show=False
)

Plot moments of the average test trajectory

In [10]:
i = np.argmin(np.linalg.norm(err - err_mean, axis=-1))
np.savetxt(path_to_saving + "/traj_index.txt", i.reshape(1), fmt="%i")

In [ ]:
_, t, n, nl = compute_linfom_err(
  path=path_to_data, index=indices[i], eval_err=False
)

In [ ]:
pp.plot_mom_evolution(
  path=path_to_saving,
  t=t,
  n_m={"FOM": n[1], "Linearized FOM": nl[1]},
  molecule=system.mix.species["molecule"],
  molecule_label="\\text{O$_2$}",
  tlim={"m0": [1e-8, 1e-4], "m1": [1e-9, 1e-4]},
  ylim_err=[1e-9,1e3],
  err_scale="log",
  hline=1e2,
  max_mom=2
)